In [1]:
# Install packages as needed:
!pip install folium
!pip install h3
!pip install branca
!pip install jenkspy

You should consider upgrading via the '/Users/forestedwards/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/forestedwards/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/forestedwards/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/forestedwards/anaconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
# IMPORTS
import pandas as pd
import folium
import h3
import jenkspy
import branca

In [3]:
class Hexagon:
    def __init__(self, name, hex_coordinate, weight, parent_loc):
        self.name = name
        self.hex_coordinate = hex_coordinate
        self.weight = weight
        self.parent_loc = parent_loc

class HexagonGrid: # Class that contains all hexagons in h3 format at some specified resolution
    def __init__(self, name, geo_locs):
        self.name = name
        self.geo_locs = geo_locs
        self.resolution = geo_locs['resolution'].iloc[0]
        self.parent_grid = None
        self.sub_grid = None
        self.hexagons = self.generate_grid()
        
    def generate_grid(self): # Returns a list of Hexagon objects
        hex_locations = [] # Keeps track of hex locations already counted
        hex_grid = [] # list of Hexagon objects to be returned
        for i in range(len(self.geo_locs)):
            location = h3.geo_to_h3(self.geo_locs.iloc[i]['latitude'], 
                                              self.geo_locs.iloc[i]['longitude'], 
                                              self.resolution)
            if location in hex_locations: # Add to weight of existing hexagon
                for item in hex_grid:
                    if location == item.hex_coordinate:
                        item.weight = item.weight + 1
            else: # Create new hexagon
                hex_locations.append(location)
                hexagon = Hexagon(self.geo_locs.iloc[i]['name'], location, 1, self.geo_locs.iloc[i]['parent'])
                hex_grid.append(hexagon)
        return hex_grid
    
    def set_sub_grid(self, sub_grid):
        # Increase weights of this grid by amount in sub_grid
        self.sub_grid = sub_grid
        while sub_grid != None:
            for i in sub_grid.hexagons:
                for j in self.hexagons:
                    if i.parent_loc == j.name:
                        j.weight = j.weight + i.weight
                        break
            sub_grid = sub_grid.sub_grid

In [4]:
def read_csv(filename, datafields): #Parameters: name of file, fields to be read
    data = pd.read_csv(filename, usecols = datafields)
    cleaned = data
    for i in range(len(data.index)): # Some values maay be null or zero, we will just drop those.
        for j in datafields:
            if pd.isnull(data.iloc[i][j]) or data.iloc[i][j] == 0:
                cleaned = cleaned.drop(i)
                break
    return cleaned

def build_color_classification(hexgrid, color_scheme):
    # Uses natural breaks classification to generate color breaks
    weights = []
    for hexagon in hexgrid.hexagons:
        weights.append(hexagon.weight)
    color_breaks = jenkspy.jenks_breaks(weights, nb_class = len(color_scheme) - 1)
    return color_breaks
    
def plot_map(grid, color_scheme):
    m = folium.Map(location = [0, 0],
                     tiles = 'Stamen Terrain', 
                     attr = "<a href=https://endless-sky.github.io/>Endless Sky</a>", 
                     zoom_start = 1, 
                     overlay = True)
    while(grid != None):
        feature_group = folium.FeatureGroup(name = grid.name, show = False)
        color_class = build_color_classification(grid, color_scheme)
        color_map = branca.colormap.LinearColormap(colors = color_scheme, index = color_class, vmin = 1)
        for hexagon in grid.hexagons:
            hex_color = color_map.rgb_hex_str(hexagon.weight)
            hex_vertices = h3.h3_to_geo_boundary(hexagon.hex_coordinate)
            folium.Polygon(locations = hex_vertices, 
                           popup = "<b>" + hexagon.name + "</b>" + ": " + str(hexagon.weight) + "<br><b>Parent: </b>" + hexagon.parent_loc, 
                           fill_color = hex_color, 
                           fill_opacity = .5,
                           weight = 1,
                           color = '#000000'
                          ).add_to(feature_group)
        feature_group.add_to(m)
        grid = grid.sub_grid
    folium.LayerControl(collapsed = False).add_to(m) # Allows toggling of resolutions while running
    return m

def generate_h3_map(filename, color_scheme):
    datafields = ['latitude', 'longitude', 'name', 'parent', 'resolution']
    nlp_geolocations = read_csv(filename, datafields)
    hex_grids_list = []
    
    # Split geolocation data into unique resolutions (16 possible resolutions): 
    for i in range(16):
        split_df = nlp_geolocations[nlp_geolocations['resolution'] == i]
        if not split_df.empty:
            hex_grid = HexagonGrid('Resolution ' + str(i), split_df)
            hex_grids_list.append(hex_grid)
    
    # Set sub grids:
    for i in range(len(hex_grids_list) - 2, -1, -1):
        hex_grids_list[i].set_sub_grid(hex_grids_list[i + 1])
        
    # Plot map:
    m = plot_map(hex_grids_list[0], color_scheme) # Color scheme: [Green, Yellow, Red]
    display(m)

In [5]:
generate_h3_map('nlp_geolocations.csv', ['#008000', '#FFFF00', '#FF0000'])